<a href="https://colab.research.google.com/github/abhilashhn1993/depression-assessment-on-twitter/blob/Dragon/Code/TweetCleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install spacy

In [0]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import spacy
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
%matplotlib inline
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

**BASIC TEXT CLEANING**
    

1.   Punctuation removal
2.   Removing URLs
3.   Removing Junk characters (non-english)
4.   Remove @ mentions from tweets
5.   Change to lower case   

In [0]:
#Method to remove texts with consecutive characters
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

In [0]:
def cleanTweets(df):
   # remove twitter Return handles (RT @xxx:)
   df['Tweets'] = np.vectorize(remove_pattern)(df['Tweets'], "RT @[\w]*:")
   # remove twitter handles (@xxx)
   df['Tweets'] = np.vectorize(remove_pattern)(df['Tweets'], "@[\w]*")
   # remove URL links (httpxxx)
   df['Tweets'] = np.vectorize(remove_pattern)(df['Tweets'], "https?://[A-Za-z0-9./]*")

   regex = re.compile('[^a-zA-Z]')
   df['Tweets'] = regex.sub(' ',df['Tweets'])

   return df

In [9]:
from google.colab import files
uploaded = files.upload()

Saving Dep_users_pre-post_all_tweets_with_senti_scores.xlsx to Dep_users_pre-post_all_tweets_with_senti_scores.xlsx


In [0]:
import io
df_pre = pd.read_excel(io.BytesIO(uploaded['Dep_users_pre-post_all_tweets_with_senti_scores.xlsx']), sheet_name='pre_tweets_all_types')
df_post = pd.read_excel(io.BytesIO(uploaded['Dep_users_pre-post_all_tweets_with_senti_scores.xlsx']), sheet_name='post_tweets_all_types')

In [78]:
df_pre.head()

,Twitter_handle,User_type,Tweets,Tweet_date,comp_score,positive_score,negative_score,neutral_score
0,doodybeardLIVE,1,ONE THOUSEY!,Wed Apr 10 23:12:12 +0000 2019,0.0000,0.000,0.000,1.000
1,doodybeardLIVE,1,yooooo that is awesome hahahah and now i want ...,Wed Apr 10 23:00:47 +0000 2019,0.8074,0.372,0.000,0.628
2,doodybeardLIVE,1,IS THAT A SONGBIRD? I have one too!!!,Wed Apr 10 22:04:51 +0000 2019,0.0000,0.000,0.000,1.000
3,doodybeardLIVE,1,Sending so much merch!!!! Yâ€™all are INCREDIB...,Wed Apr 10 21:56:22 +0000 2019,0.7526,0.352,0.000,0.648
4,doodybeardLIVE,1,I apologize for whoever that shit person was ;...,Wed Apr 10 20:54:44 +0000 2019,-0.1280,0.208,0.183,0.609


In [0]:
df_pre['Tweets'] = df_pre['Tweets'].map(lambda x: str(x))

In [82]:
df_pre_conc = df_pre.groupby(['Twitter_handle','User_type'])['Tweets'].agg(' '.join).reset_index(name='Tweets')
df_pre_conc.head()

,Twitter_handle,User_type,Tweets
0,1racy,3,Are they being announced on the 22nd? My third...
1,300Stone,3,Depends what music is playing When Iâ€™m on th...
2,44Hayley_,1,Insurance made my meds go from $300+ to $130. ...
3,4allofusclub,2,We are recruiting for a Delegate Manager to as...
4,6llvck,2,sure jk fuck yeah we missed ...


In [0]:
df_pre_conc['Tweets'] = df_pre_conc['Tweets'].map(lambda x: str(x))

In [0]:
df_pre_conc = cleanTweets(df_pre_conc)

In [0]:
#Remove non-alphanumeric values
for index,row in df_pre_conc.iterrows():
  out = regex.sub(' ',row['Tweets'])
  df_pre_conc.at[index, 'Tweets'] = ' '.join(out.split())

In [86]:
df_pre_conc.head()

,Twitter_handle,User_type,Tweets
0,1racy,3,Are they being announced on the nd My third on...
1,300Stone,3,Depends what music is playing When I m on the ...
2,44Hayley_,1,Insurance made my meds go from to If I m payin...
3,4allofusclub,2,We are recruiting for a Delegate Manager to as...
4,6llvck,2,sure jk fuck yeah we missed u focus on the les...
